In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd

f = open('comidaPeruanaEntidadesTreinamento.txt', "rw")
i = 0
for line in f:
    if line == '\n': 
        print line
        line = 'o \t 0\n'
        for i in range(i, 15):
            line = line + line
        line = line+ '\n'
        f.write(line)
    else : i = i+1
f.close()

              0            1
0          cómo            O
1            se            O
2       prepara            O
3            un            O
4         plato            O
5            de            O
6           ají        PLATO
7            de        PLATO
8       gallina        PLATO
9        cuánto            O
10       cuesta            O
11           un            O
12        plato            O
13           de            O
14          cau        PLATO
15          cau        PLATO
16        quién            O
17      inventó            O
18           el            O
19     aguadito        PLATO
20            a            O
21      cuántos            O
22       grados            O
23           se            O
24         debe            O
25    cocinarse            O
26           el            O
27      chancho  INGREDIENTE
28         cuál            O
29           es            O
...         ...          ...
1497   preparar            O
1498    soltero        PLATO
1499         d

In [11]:

x = pd.read_csv('comidaPeruanaEntidadesTreinamento.txt',sep="\t",header=None)

print (x)


In [4]:

# Data settings.
num_examples = 1527
num_words = 15
num_features = 2
num_tags = 3

# Random features.
x = np.random.rand(num_examples, num_words, num_features).astype(np.float32)

# Random tag indices representing the gold sequence.
y = np.random.randint(num_tags, size=[num_examples, num_words]).astype(np.int32)

# All sequences in this example have the same length, but they can be variable in a real model.
sequence_lengths = np.full(num_examples, num_words - 1, dtype=np.int32)

# Train and evaluate the model.
with tf.Graph().as_default():
  with tf.Session() as session:
    # Add the data to the TensorFlow graph.
    x_t = tf.constant(x)
    y_t = tf.constant(y)
    sequence_lengths_t = tf.constant(sequence_lengths)

    # Compute unary scores froriable("weights", [num_features, num_tags])
    matricized_x_t = tfriable("weights", [num_features, num_tags])
    matricized_x_t = tfm a linear layer.
    weights = tf.get_variable("weights", [num_features, num_tags])
    matricized_x_t = tf.reshape(x_t, [-1, num_features])
    matricized_unary_scores = tf.matmul(matricized_x_t, weights)
    unary_scores = tf.reshape(matricized_unary_scores,
                              [num_examples, num_words, num_tags])

    # Compute the log-likelihood of the gold sequences and keep the transition
    # params for inference at test time.
    log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
        unary_scores, y_t, sequence_lengths_t)

    # Compute the viterbi sequence and score.
    viterbi_sequence, viterbi_score = tf.contrib.crf.crf_decode(
        unary_scores, transition_params, sequence_lengths_t)

    # Add a training op to tune the parameters.
    loss = tf.reduce_mean(-log_likelihood)
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    session.run(tf.global_variables_initializer())

    mask = (np.expand_dims(np.arange(num_words), axis=0) <
            np.expand_dims(sequence_lengths, axis=1))
    total_labels = np.sum(sequence_lengths)

    # Train for a fixed number of iterations.
    for i in range(1000):
      tf_viterbi_sequence, _ = session.run([viterbi_sequence, train_op])
      if i % 100 == 0:
        correct_labels = np.sum((y == tf_viterbi_sequence) * mask)
        accuracy = 100.0 * correct_labels / float(total_labels)
        print("Accuracy: %.2f%%" % accuracy)

Accuracy: 17.89%
Accuracy: 58.95%
Accuracy: 71.58%
Accuracy: 76.84%
Accuracy: 78.42%
Accuracy: 81.58%
Accuracy: 82.63%
Accuracy: 83.68%
Accuracy: 84.21%
Accuracy: 86.84%
